# Xploration Notebook

In this notebook we will build a first method to find the frequency of words in a corpus and see their relevance.

We will work with the corpus of EHR (CSV files - Lounes), and then build the TF and then IDF model.

WE will then try to have visualization aspect.

In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
# Download the csv file to analyze
TEST = {1: "le tf-idf de l'anglais term frequency-inverse document frequency est une méthode de pondération souvent utilisée en recherche d'information et en particulier dans la fouille de textes",
       2: "cette mesure statistique permet d'évaluer l'importance d'un terme contenu dans un document relativement à une collection ou un corpus",
       3: "le poids augmente proportionnellement au nombre d'occurrences du mot dans le document",
       4: "il varie également en fonction de la fréquence du mot dans le corpus ainsi le",
       5: "des variantes de la formule originale sont souvent utilisées dans des moteurs de recherche pour apprécier la pertinence d'un document en fonction des critères de recherche de l'utilisateur"}
df = pd.DataFrame.from_dict(TEST, orient = 'index')
#df[1] = [element[0].split() for _, element in df.iterrows()]
all_words = set([word for _, words in df.iterrows() for word in words[0].split() for _, words in df.iterrows()])
all_words_dict = {word:0 for word in all_words}
#df

## Bag of words

### Basic Implementation

In [4]:
all_words_dict = {word: 0 for word in all_words}
all_words_dict = {key: value for key, value in sorted(all_words_dict.items())}
sentences_dict = {i:all_words_dict.copy() for i in list(TEST.keys())}

for k, sentence in TEST.items():
    for word in sentence.split():
        sentences_dict[k][word] +=1
        
X_bow = []
for i in list(TEST.keys()):
    X_bow.append(list(sentences_dict[i].values()))

df_bow = pd.DataFrame(X_bow, columns = [i for i in list(all_words_dict.keys())])
df_bow

,ainsi,apprécier,au,augmente,cette,collection,contenu,corpus,critères,d'information,...,textes,tf-idf,un,une,utilisée,utilisées,variantes,varie,à,également
0,0,0,0,0,0,0,0,0,0,1,...,1,1,0,1,1,0,0,0,0,0
1,0,0,0,0,1,1,1,1,0,0,...,0,0,2,1,0,0,0,0,1,0
2,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,1
4,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,1,1,0,0,0


In [ ]:
def raw_count_text(doc):
    """
    Given a text, return a dictionnary with the amount of occurences of a word
    """
    words = {}
    for w in doc.split():
        if w not in words:
            words[w] = 1
        else:
            words[w]+=1
    return words

dict_df = {i:{} for i in range(len(TEST))}
i=0
for k, sentence in TEST.items():
    dict_df[i] = raw_count_text(sentence)
    i+=1

df  = pd.DataFrame.from_dict(dict_df, orient = 'index')
df

X_home = []
for idx, row in df.iterrows():
    a = list(row)
    a = [0 if math.isnan(num) else int(num) for num in a]
    X_home.append(a)
X_home = np.array(X_home)
df_CV = pd.DataFrame(X_home, columns = [i for i in range(len(X_home[0]))])
df_CV

### Python Library

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(list(TEST.values()))
X = X.toarray()
X
df_CV = pd.DataFrame(X, columns = [i for i in range(len(X[0]))])
df_CV

## TF aspect

In [ ]:
# Finding the count in each document
def raw_count_text(doc):
    """
    Given a text, return a dictionnary with the amount of occurences of a word
    """
    words = {}
    for w in doc.split():
        if w not in words:
            words[w] = 1
        else:
            words[w]+=1
    return words
    
def term_frequency_text(doc, log = False):
    """
    Given a text, return a dictionnary with the frequency of a word within the text
    """
    words = {}
    for w in doc.split():
        if w not in words:
            words[w] = 1
        else:
            words[w] +=1
    if log == True:
        words = {k:np.log(1+v) for k,v in words.items()}
    else:
        words = {k:(v*100)/sum(words.values()) for k,v in words.items()}
    return words

def bool_frequency(doc):
    words = {}
    for w in all_words:
        if w in doc.split():
            words[w] = 1
        else:
            words[w] = 0
    return words

# Finding the count for the whole corpus
def raw_count_corpus(data):
    """
    Given a data frame composed of text, return a dictionnary with the amount of occurences of a word
    """
    words = {}
    for _, string in data.iterrows():
        for w in string[0].split():
            if w not in words:
                words[w] = 1
            else:
                words[w]+=1
    return words
    
def term_frequency_corpus(data, log = False):
    """
    Given a dataframe composed of texts, return a dictionnary with the frequency of a word within the corpus
    """
    words = {}
    for _, string in data.iterrows():
        for w in string[0].split():
            if w not in words:
                words[w] = 1
            else:
                words[w] +=1
    if log == True:
        words = {k:np.log(1+v) for k,v in words.items()}
    else:
        words = {k:(v*100)/sum(words.values()) for k,v in words.items()}
    return words

## IDF aspect

In [ ]:
# Compute the IDF
def idf(data):
    N = data.shape[0]
    frequency = {}
    for _, word in data.iterrows():
        bf = bool_frequency(word[0])
        for k,v in bf.items():
            if v == 1:
                if k not in frequency:
                    frequency[k] = 1
                else:
                    frequency[k] +=1
    return frequency


## TF-IDF together

In [ ]:
tf_idf = {}
idf_data = idf(df)
tfidf = {i:{word:0 for word, _ in idf_data.items()} for i in range(df.shape[0])}
i=0
for _, text in df.iterrows():
    a = term_frequency_text(text[0])
    for word, freq in a.items():
        tfidf[i][word] = freq * idf_data[word]
    i+=1
X_home_tfidf = pd.DataFrame.from_dict(tfidf, orient = 'index')

## TF-IDF Vectorizer

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
corpus = list(TEST.values())

In [ ]:
corpus

In [ ]:
vectorizer = TfidfVectorizer()

In [ ]:
X = vectorizer.fit_transform(corpus)

In [ ]:
vectorizer.get_feature_names_out()

In [ ]:
X_tfidf = X.toarray()
df_tfidf = pd.DataFrame(X_tfidf, columns = [i for i in range(len(X_tfidf[0]))])
df_tfidf


In [ ]:
X